# Liquid Neural Network (LTC based)

## Scratch Implementation (Pytorch)

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim

In [11]:
class LTCCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LTCCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        # Gate mechanism - func f: Input I(t), hidden_state x(t)
        self.f = nn.Linear(in_features=input_size + hidden_size, out_features=hidden_size)

        # parameters
        # Tau (Base time-constant)
        # A (Bias)
        self.tau = nn.Parameter(torch.ones(hidden_size))
        self.A = nn.Parameter(torch.ones(hidden_size))

    def forward(self, input_data, h_state, *, dt=1.0):
        combined = torch.cat([input_data, h_state], dim=1)

        self.gate = torch.sigmoid(self.f(combined))

        # Closed-form update (implicit step)
        # numerator pushes X towards A based upon input strength
        num = h_state + dt * self.gate * self.A
        # denominator stabilities the X & prevents explosion.
        den = 1 + dt * (1/self.tau + self.gate)
        update_state = num / den

        return update_state
        
        

### checks

In [12]:
ltc_model = LTCCell(input_size=5, hidden_size=10)
x = torch.randn(3, 5)
h = torch.zeros(3, 10)
print(f"Input shape: {x.shape}")
print(f"Input sample: {x[: :2]}")

h_new = ltc_model(x, h, dt=0.5)
print(f"Output shape: {h_new.shape}")
print(f"Output sample: {h_new[:, :2]}")

Input shape: torch.Size([3, 5])
Input sample: tensor([[-0.5512, -0.5881, -0.9956,  0.5971,  0.1927],
        [ 1.2109,  0.0074,  0.7860,  2.3019, -0.2146]])
Output shape: torch.Size([3, 10])
Output sample: tensor([[0.1539, 0.1580],
        [0.1355, 0.1607],
        [0.1740, 0.1433]], grad_fn=<SliceBackward0>)
